# 从表里查询数据

In [ ]:
# 全部数据
SELECT * FROM register_user

# 指定列
SELECT user_email, user_id FROM register_user

# 指定其他条件
SELECT * FROM register_user WHERE user_id =1

# 更新、删除表里的数据

In [ ]:
UPDATE register_user 
SET user_name = 'helloguy'
WHERE user_id = 1;

# 删除数据
DELETE FROM register_user
WHERE user_id is NULL;

# autocommit、commit、rollback

In [ ]:
# 停用自动提交
SET AUTOCOMMIT =OFF;

# 删除表内容后，还可以回退
SELECT * FROM register_user;
DELETE FROM register_user;
-- ROLLBACK;

# 通过commit提交，就不能回退了
COMMIT;

# current_date & current_time

In [ ]:
# 建好表
CREATE TABLE test_datetime(
	my_date DATE,
	my_time TIME,
	my_datetime DATETIME
);

# 基于事件函数插入一条row
INSERT INTO test_datetime
VALUES (CURRENT_DATE(), CURRENT_TIME(), NOW());
SELECT * FROM test_datetime;

# 时间函数可以修改
INSERT INTO test_datetime
VALUES (CURRENT_DATE() -1 , CURRENT_TIME()-600, NOW());
SELECT * FROM test_datetime;

# unique constraint 唯一约束
确保每个都不一样


In [ ]:
-- 创建一个表，里面的id字段是唯一的
CREATE TABLE test_unique(
	id INT UNIQUE,
	name VARCHAR(25)
);

-- 修改之前已经创建好的表，把某一列改成unique constraint
ALTER TABLE test_unique
ADD CONSTRAINT UNIQUE (name);

-- 试试插入两条数据，不一样的id，不一样的name，没问题
INSERT INTO test_unique
VALUES (1, 'hello'), (2, 'world');

-- 再试试插入两条数据，不一样的id，但是name一样，会报错
INSERT INTO test_unique
VALUES (3, 'hello');

# NOT NULL constraint 非空约束

In [ ]:
-- 创建一个表
CREATE TABLE test_not_null_constraint(
	test_id INT UNIQUE,
	test_name VARCHAR(25) NOT NULL
);

-- 给已经创建的表的列设置not null constraint
-- MODIFY 用于修改列的数据类型、是否 NOT NULL、默认值等列属性；
-- UNIQUE 是索引/约束，需要通过 ADD UNIQUE / ADD CONSTRAINT / CREATE UNIQUE INDEX 来添加
ALTER TABLE test_not_null_constraint
MODIFY COLUMN test_id NOT NULL;

-- 试着插入一个test_name为空的记录，会报错
INSERT INTO test_not_null_constraint
VALUES (1, NULL);

# check constraint 检查约束

In [ ]:

-- 创建一个表，带有check constraint
CREATE TABLE test_check_constraint(
	test_id INT,
	test_age INT,
	-- CONSTRAINT 用来标记这个check的名字，如果不写也行
	CONSTRAINT chk_age CHECK (test_age >= 18)
	-- CHECK (test_age >= 18
)

-- 在已有的表里添加check constraint
ALTER TABLE test_check_constraint
ADD CONSTRAINT chk_id CHECK (test_id >= 10);


-- 试试看能否触发check constraint
INSERT INTO test_check_constraint
VALUES (1, 10);  

-- 删除表里已有的check constraint
ALTER TABLE test_check_constraint
DROP CONSTRAINT chk_age;

-- 这次就不会触发check constraint
INSERT INTO test_check_constraint
VALUES (1, 10);  

# default constraint 默认约束
设置某一列，默认采用xxx值

In [ ]:
-- 创建一个表，带有default constraint
CREATE TABLE test_default_constraint(
	test_id INT,
	test_price INT DEFAULT 10
);

-- 在已有的表里添加default constraint
ALTER TABLE test_default_constraint
ALTER COLUMN test_id SET DEFAULT 1;

-- 插入数据时，不指定test_price列，就会默认为10
INSERT INTO test_default_constraint (test_id)
VALUES (1),
	   (2);

# primary key 主键约束
将某一列变成主键，比如身份证号

一个表里只能有一个主键



In [ ]:
-- 新建一个有primary key的表
CREATE TABLE test_primary_key(
	id INT PRIMARY KEY,
	name VARCHAR(25)
);

-- 把某个列变成primary key，直接这么操作的话应该会报错
-- 因为前面已经设置好id为primary key
ALTER TABLE test_primary_key
ADD CONSTRAINT  PRIMARY KEY (name);

INSERT INTO test_primary_key
VALUES (1, 'hello'),
	   (2, 'world');


# 自动增长auto_increment
设置好以后，每次新增一个数据，primary key自动递增

In [ ]:
-- auto_increment 只能给primary key设置
CREATE TABLE test_auto_increment(
	id INT PRIMARY KEY AUTO_INCREMENT ,
	name VARCHAR(25)
);

-- 测试插入一些数据
INSERT INTO test_auto_increment (name)
VALUES ('first'),
	   ('second'),
	   ('third');

SELECT * FROM test_auto_increment;

-- 设置auto_increment的起始值从100开始，再插入试试看
ALTER TABLE test_auto_increment
AUTO_INCREMENT = 100;

# 外键 foreign key
一个表可以用另一个表的数据
